<br>
<br>
<br>
<br>

# Do the same project(pipeline- (01 & 02))

<br>
<br>
<br>
<br>


In [121]:

import pandas as pd 
import numpy as np 
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier



In [122]:
df = pd.read_csv("dataset/train.csv")
df.sample(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
266,267,0,3,"Panula, Mr. Ernesti Arvid",male,16.0,4,1,3101295,39.6875,NaN,S
801,802,1,2,"Collyer, Mrs. Harvey (Charlotte Annie Tate)",female,31.0,1,1,C.A. 31921,26.2500,NaN,S


In [123]:
df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)
df.sample(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
850,0,3,male,4.0,4,2,31.275,S
642,0,3,female,2.0,3,2,27.900,S


In [124]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

<br>

## 01. train_test_split()

In [125]:

X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df["Survived"],test_size=0.2,random_state=42)

X_train.sample(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
861,2,male,21.0,1,0,11.5000,S
656,3,male,NaN,0,0,7.8958,S


In [126]:
y_train.sample(2)

515    0
9      1
Name: Survived, dtype: int64

In [127]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(712, 7)
(712,)
(179, 7)
(179,)


In [128]:
# imputation transformer
trnf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')


In [129]:
# one hot encoding

trnf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')


In [130]:
#(in this we use feature selection that why we need to use minmaxscaler)
# scaling:

trnf3 = ColumnTransformer(
    transformers=[
        ('scale',MinMaxScaler(),slice(0,10)) # after applying trn1,trns1 we have total 9 columns:
    ]
)

# we apply scaling all the column so no need for remider, if we give then i get an error:


<br>

# feature selection:

<br>

In [131]:
from sklearn.feature_selection import SelectKBest,chi2

In [132]:

trnf4 = SelectKBest(score_func=chi2,k=8)  #  (k = 8 top 8 important columns:)


In [133]:
# train the model: 

trnf5 = DecisionTreeClassifier()

<br>
<br>
<br>

# Create Pipelines:

<br>
<br>
<br>

In [134]:
from sklearn.pipeline import Pipeline,make_pipeline

In [135]:
pipe = Pipeline(
    # transformer name, and hers object name
    [
        ('trnf1',trnf1),
        ('trnf2',trnf2),
        ('trnf3',trnf3),
        ('trnf4',trnf4),
        ('trnf5',trnf5),
     ]
)

In [136]:
# we can also do the same thing with make_pipeline:

pipe  = make_pipeline(trnf1,trnf2,trnf3,trnf4,trnf5)

In [137]:
# train the pipe line:

pipe.fit(X_train,y_train)


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x760d9f91ccc0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

<br>
<br>
<br>

# Explore the pipeLine:

<br>
<br>
<br>

In [138]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x760d9f91ccc0>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [139]:
pipe.named_steps['columntransformer-1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [140]:
pipe.named_steps['columntransformer-1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder',
  FunctionTransformer(accept_sparse=True, check_inverse=False,
                      feature_names_out='one-to-one'),
  [0, 1, 3, 4, 5])]

In [141]:
pipe.named_steps['columntransformer-1'].transformers_[0]

('impute_age', SimpleImputer(), [2])

In [142]:
pipe.named_steps['columntransformer-1'].transformers_[0][1]

SimpleImputer()

In [143]:
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_ # mean value

array([29.49884615])

In [144]:
pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [145]:
y_pred = pipe.predict(X_test)
accuracy_score(y_test,y_pred)*100

62.56983240223464

<br>
<br>
<br>

# cross validation Using Pipeline:

<br>
<br>
<br>

In [146]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy') 


array([0.6013986 , 0.62237762, 0.68309859, 0.65492958, 0.63380282])

<br>
<br>
<br>

# Exporting the model:

<br>
<br>
<br>

In [147]:
import pickle

pickle.dump(pipe,open('model/with_pipeline.pkl','wb'))

